In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
!pip install pandas nltk regex openpyxl

In [22]:
import os
import pandas as pd
import nltk
import regex as re
from nltk.tokenize import WordPunctTokenizer
# Download tokenizer
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
# Path to your ingested CSV in the repo
df = pd.read_csv("https://raw.githubusercontent.com/lhiwi/ethioMart-ner-project/task-2/data/raw/telegram_data.csv")
print("Total messages:", len(df))

# Show first few rows
df.head()


Total messages: 2500


,date,sender,raw,views,channel,cleaned
0,2025-06-21 06:32:27+00:00,-1001195361398,💻 Dell Precision Workstation\n\nA powerful per...,716.0,@nevacomputer,ለአጭር መልእክት\n ይደዉሉ
1,2025-06-21 06:32:27+00:00,-1001195361398,NaN,716.0,@nevacomputer,NaN
2,2025-06-21 06:32:27+00:00,-1001195361398,NaN,716.0,@nevacomputer,NaN
3,2025-06-21 06:32:27+00:00,-1001195361398,NaN,618.0,@nevacomputer,NaN
4,2025-06-11 13:56:52+00:00,-1001195361398,NaN,1698.0,@nevacomputer,NaN


In [18]:
# Sample 50 non-empty cleaned messages
sample_df = df["cleaned"].dropna().sample(50, random_state=42).reset_index(drop=True)
print("Sampled messages:")
sample_df


Sampled messages:


,cleaned
0,አድራሻሜክሲኮ ኮሜርስ ጀርባ መዚድ ፕላዛ የመጀመሪያ ደረጃ እንደወጡ የቢ...
1,አድራሻሜክሲኮ ኮሜርስ ጀርባ መዚድ ፕላዛ የመጀመሪያ ደረጃ እንደወጡ የቢ...
2,በተጨማሪ ከብር በላይ የሆኑ\nሁለትዕቃዎች በአንድዲሊቨሪ ግዜ ሲያዙ ስጦ...
3,አድራሻሜክሲኮ ኮሜርስ ጀርባ መዚድ ፕላዛ የመጀመሪያ ደረጃ እንደወጡ የቢ...
4,አድራሻሜክሲኮ ኮሜርስ ጀርባ መዚድ ፕላዛ የመጀመሪያ ደረጃ እንደወጡ የቢ...
5,በአየር ደናሹ ሮቦት\n\nልጆች የአየር ግፊትንና አካሄድን በተግባር የሚማ...
6,ተወዳጆቹን ካራክተሮች ለልጆችዎ\n\nየስፓይደር ማን እና ሱፐርማን \nአሻ...
7,አና በመነሻዬ ትምህርታዊ መጫዎቻዎች ተማርካ ፀቧን ረስታለች \n\nመነሻዬ...
8,ተወዳጆቹን ካራክተሮች ለልጆችዎ\n\nየስፓይደር ማን እና ሱፐርማን \nአሻ...
9,መምህርት ዙፊት ስለ ቀለሜ መነሻዬ ጥቅል ምን ትላለች


# Tokenize & CRreate a CoNLL Template

In [23]:
# Tokenize using NLTK's WordPunctTokenizer and build CoNLL template
tokenizer = WordPunctTokenizer()

def tokenize_nltk(text: str) -> list[str]:
    """
    Use NLTK's WordPunctTokenizer to split text into tokens.
    This doesn't require Punkt downloads.
    """
    return tokenizer.tokenize(text)

lines = []
for msg in sample_df:
    tokens = tokenize_nltk(msg)
    for tok in tokens:
        lines.append(f"{tok}\tO")   # default label O
    lines.append("")  # blank line between messages

# Write out the template
with open("labeling_template.conll", "w", encoding="utf8") as f:
    f.write("\n".join(lines))

print(" Wrote labeling_template.conll with", len(sample_df), "messages.")


 Wrote labeling_template.conll with 50 messages.


In [24]:
from google.colab import files
files.download('labeling_template.conll')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
# Verify your annotated CoNLL file from Drive

def read_conll(path):
    docs = []
    with open(path, encoding="utf8") as f:
        doc = []
        for line in f:
            line = line.strip()
            if not line:
                if doc:
                    docs.append(doc)
                    doc = []
            else:
                token, label = line.split("\t")
                doc.append((token, label))
        if doc:
            docs.append(doc)
    return docs

annotated = read_conll('/content/drive/MyDrive/labeling_template.conll')

print("Labeled messages:", len(annotated))
print("First message labels:")
for token, label in annotated[0]:
    print(f"{token}\t{label}")


Labeled messages: 50
First message labels:
አድራሻሜክሲኮ	O
ኮሜርስ	O
ጀርባ	O
መዚድ	O
ፕላዛ	O
የመጀመሪያ	O
ደረጃ	O
እንደወጡ	O
የቢሮ	O
ቁጥር	O
ያገኙናል	O
